# ***File dùng để Upsert các Vector đã được lưu ngoài file JSON lên PINECONE***

In [10]:
#Tạo đường dẫn chung để đọc utils
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

In [11]:
# Tải các thư viện cần thiết
import json
from pinecone.grpc import PineconeGRPC as Pinecone
from dotenv import load_dotenv
import os
from utils.load_chunks_json import load_chunks_from_json

In [12]:
# Khởi tạo các biến cần thiết
load_dotenv("../.env")

PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
HOST_DENSE = os.getenv("HOST_DENSE")
HOST_SPARSE = os.getenv("HOST_SPARSE")

In [13]:
# Đọc Dense và Sparse của môn Triết Học
TrietHoc_dense_vector = load_chunks_from_json(r"./TrietHoc/TrietHoc_Dense_Global.json")
TrietHoc_sparse_vector = load_chunks_from_json(r"./TrietHoc/TrietHoc_Sparse_Global.json")

# Đọc Dense và Sparse của môn Lịch Sử Đảng
LSD_dense_vector = load_chunks_from_json(r"./LichSuDang/Lich_Su_Dang_Dense_Global.json")
LSD_sparse_vector = load_chunks_from_json(r"./LichSuDang/Lich_Su_Dang_Sparse_Global.json")

# Đọc Child chunks cho Parent Document Retrieval
TrietHoc_child_dense_vector = load_chunks_from_json(r"./TrietHoc/TrietHoc_Child_Dense_Optimal.json")
LSD_child_dense_vector = load_chunks_from_json(r"./LichSuDang/Lich_Su_Dang_Child_Dense_Optimal.json")

print(f"SỐ LƯỢNG CHUNKS ĐÃ TẢI:")
print(f"  TrietHoc Dense: {len(TrietHoc_dense_vector)}")
print(f"  TrietHoc Sparse: {len(TrietHoc_sparse_vector)}")
print(f"  TrietHoc Child Dense: {len(TrietHoc_child_dense_vector)}")
print(f"  LSD Dense (Parent): {len(LSD_dense_vector)}")
print(f"  LSD Sparse: {len(LSD_sparse_vector)}")
print(f"  LSD Child Dense: {len(LSD_child_dense_vector)}")

# Kiểm tra cấu trúc child chunk - Triết học
if TrietHoc_child_dense_vector:
    sample_child = TrietHoc_child_dense_vector[0]
    print(f"\nCẤU TRÚC CHILD CHUNK - TRIẾT HỌC:")
    print(f"  ID: {sample_child['id']}")
    print(f"  Parent ID: {sample_child['metadata'].get('parent_id', 'N/A')}")
    print(f"  Content length: {len(sample_child['metadata']['content'])} chars")
    print(f"  Vector dimension: {len(sample_child['values'])}")

# Kiểm tra cấu trúc child chunk - Lịch sử Đảng
if LSD_child_dense_vector:
    sample_child = LSD_child_dense_vector[0]
    print(f"\nCẤU TRÚC CHILD CHUNK - LỊCH SỬ ĐẢNG:")
    print(f"  ID: {sample_child['id']}")
    print(f"  Parent ID: {sample_child['metadata'].get('parent_id', 'N/A')}")
    print(f"  Content length: {len(sample_child['metadata']['content'])} chars")
    print(f"  Vector dimension: {len(sample_child['values'])}")

SỐ LƯỢNG CHUNKS ĐÃ TẢI:
  TrietHoc Dense: 400
  TrietHoc Sparse: 400
  TrietHoc Child Dense: 1819
  LSD Dense (Parent): 347
  LSD Sparse: 347
  LSD Child Dense: 1971

CẤU TRÚC CHILD CHUNK - TRIẾT HỌC:
  ID: TrietHoc_chunk_00000_child_0
  Parent ID: TrietHoc_chunk_00000
  Content length: 213 chars
  Vector dimension: 1024

CẤU TRÚC CHILD CHUNK - LỊCH SỬ ĐẢNG:
  ID: LSD_chunk_00000_child_0
  Parent ID: LSD_chunk_00000
  Content length: 320 chars
  Vector dimension: 1024


In [14]:
pc = Pinecone(api_key=PINECONE_API_KEY)

# Lấy ra Index của Dense Database
dense_index = pc.Index(host=HOST_DENSE)

# Lấy ra Index của Sparse Database
sparse_index = pc.Index(host=HOST_SPARSE)

## ***Upsert môn Triết Học***

## Hybrid search

In [15]:
dense_index.upsert(
  vectors = TrietHoc_dense_vector,
  namespace="triet-hoc"
)
sparse_index.upsert(
  vectors = TrietHoc_sparse_vector,
  namespace="triet-hoc"
)
print("Upsert completed successfully.")

Upsert completed successfully.


## ***Upsert môn Lịch Sử Đảng***

## Hybrid search

In [16]:
dense_index.upsert(
  vectors=LSD_dense_vector,
  namespace="lich-su-dang"
)
sparse_index.upsert(
  vectors=LSD_sparse_vector,
  namespace="lich-su-dang"
)
print("Upsert LSD completed successfully.")

Upsert LSD completed successfully.


## ***Children upsert cho Parent document retrieval***

## Lịch sử Đảng

In [17]:
# DEBUG UPSERT PROCESS - LỊCH SỬ ĐẢNG
print("DEBUG UPSERT PROCESS - LỊCH SỬ ĐẢNG")
print("="*50)

# Test upsert một chunk đơn lẻ
if LSD_child_dense_vector:
    print(f"\nTEST UPSERT 1 CHUNK - LSD:")
    try:
        test_chunk = [LSD_child_dense_vector[0]]  # Chỉ lấy 1 chunk để test
        response = dense_index.upsert(
            vectors=test_chunk,
            namespace="lich-su-dang-children-test"  # Test namespace riêng
        )
        print(f"SUCCESS: Test upsert thành công: {response}")
    except Exception as e:
        print(f"ERROR: Test upsert thất bại: {e}")
        print(f"Error type: {type(e)}")
        import traceback
        print(f"Traceback: {traceback.format_exc()}")

# Kiểm tra namespace hiện tại
print(f"\nNAMESPACE HIỆN TẠI:")
try:
    stats = dense_index.describe_index_stats()
    for namespace, info in stats.namespaces.items():
        print(f"  {namespace}: {info.vector_count} vectors")
except Exception as e:
    print(f"ERROR: Không thể lấy stats: {e}")

DEBUG UPSERT PROCESS - LỊCH SỬ ĐẢNG

TEST UPSERT 1 CHUNK - LSD:
SUCCESS: Test upsert thành công: upserted_count: 1


NAMESPACE HIỆN TẠI:
  lich-su-dang-children-test: 0 vectors
  lich-su-dang: 347 vectors
  triet-hoc: 400 vectors
  lich-su-dang-children: 450 vectors


In [18]:
# UPSERT VỚI BATCH SIZE NHỎ VÀ ERROR HANDLING - LỊCH SỬ ĐẢNG
print("UPSERT CHILD CHUNKS VỚI BATCH SIZE NHỎ - LỊCH SỬ ĐẢNG")
print("="*50)

# Xóa namespace test trước
try:
    dense_index.delete(delete_all=True, namespace="lich-su-dang-children-test")
    print("SUCCESS: Đã xóa test namespace")
except:
    pass

# Xóa namespace chính để bắt đầu fresh
try:
    dense_index.delete(delete_all=True, namespace="lich-su-dang-children")
    print("SUCCESS: Đã xóa namespace 'lich-su-dang-children'")
except Exception as e:
    print(f"Warning: {e}")

print(f"Bắt đầu upsert {len(LSD_child_dense_vector)} chunks...")

# Upsert với batch size nhỏ hơn
batch_size = 50  # Giảm từ 100 xuống 50
total_upserted = 0
failed_batches = []

import time

for i in range(0, len(LSD_child_dense_vector), batch_size):
    batch = LSD_child_dense_vector[i:i+batch_size]
    batch_num = i // batch_size + 1
    total_batches = (len(LSD_child_dense_vector) + batch_size - 1) // batch_size
    
    try:
        print(f"Upsert batch {batch_num}/{total_batches} ({len(batch)} chunks)...", end=" ")
        
        # Upsert batch
        response = dense_index.upsert(
            vectors=batch,
            namespace="lich-su-dang-children"
        )
        
        total_upserted += response.upserted_count
        print(f"SUCCESS: {response.upserted_count}")
        
        # Kiểm tra nếu không upsert được hết
        if response.upserted_count != len(batch):
            print(f"  WARNING: Expected {len(batch)}, got {response.upserted_count}")
            failed_batches.append(batch_num)
        
        # Thêm delay nhỏ để tránh rate limiting
        time.sleep(0.1)
        
    except Exception as e:
        print(f"ERROR: {e}")
        failed_batches.append(batch_num)
        
        # Thử upsert từng chunk một trong batch bị lỗi
        print(f"  Trying individual chunks in batch {batch_num}...")
        for j, chunk in enumerate(batch):
            try:
                individual_response = dense_index.upsert(
                    vectors=[chunk],
                    namespace="lich-su-dang-children"
                )
                total_upserted += individual_response.upserted_count
                if j % 10 == 0:  # Print progress mỗi 10 chunks
                    print(f"    Chunk {j+1}/{len(batch)}")
            except Exception as individual_error:
                print(f"    ERROR: Failed chunk {j+1}: {individual_error}")

print(f"\nKẾT QUẢ - LỊCH SỬ ĐẢNG:")
print(f"Total upserted: {total_upserted}/{len(LSD_child_dense_vector)}")
print(f"Success rate: {total_upserted/len(LSD_child_dense_vector)*100:.1f}%")

if failed_batches:
    print(f"Failed batches: {failed_batches}")
else:
    print("SUCCESS: All batches successful!")

# Kiểm tra kết quả cuối cùng
print(f"\nKIỂM TRA NAMESPACE:")
stats = dense_index.describe_index_stats()
for namespace, info in stats.namespaces.items():
    if 'lich-su-dang-children' in namespace:
        print(f"  {namespace}: {info.vector_count} vectors")

if total_upserted == len(LSD_child_dense_vector):
    print(f"\nHOÀN THÀNH! Đã upsert thành công tất cả {total_upserted} chunks!")
else:
    missing = len(LSD_child_dense_vector) - total_upserted
    print(f"\nWARNING: Thiếu {missing} chunks. Có thể cần retry.")

UPSERT CHILD CHUNKS VỚI BATCH SIZE NHỎ - LỊCH SỬ ĐẢNG
SUCCESS: Đã xóa test namespace
SUCCESS: Đã xóa namespace 'lich-su-dang-children'
Bắt đầu upsert 1971 chunks...
Upsert batch 1/40 (50 chunks)... SUCCESS: 50
Upsert batch 2/40 (50 chunks)... SUCCESS: 50
Upsert batch 3/40 (50 chunks)... SUCCESS: 50
Upsert batch 4/40 (50 chunks)... SUCCESS: 50
Upsert batch 5/40 (50 chunks)... SUCCESS: 50
Upsert batch 6/40 (50 chunks)... SUCCESS: 50
Upsert batch 7/40 (50 chunks)... SUCCESS: 50
Upsert batch 8/40 (50 chunks)... SUCCESS: 50
Upsert batch 9/40 (50 chunks)... SUCCESS: 50
Upsert batch 10/40 (50 chunks)... SUCCESS: 50
Upsert batch 11/40 (50 chunks)... SUCCESS: 50
Upsert batch 12/40 (50 chunks)... SUCCESS: 50
Upsert batch 13/40 (50 chunks)... SUCCESS: 50
Upsert batch 14/40 (50 chunks)... SUCCESS: 50
Upsert batch 15/40 (50 chunks)... SUCCESS: 50
Upsert batch 16/40 (50 chunks)... SUCCESS: 50
Upsert batch 17/40 (50 chunks)... SUCCESS: 50
Upsert batch 18/40 (50 chunks)... SUCCESS: 50
Upsert batch 19/

## Triết học

In [19]:
# DEBUG UPSERT ERROR
print("DEBUG UPSERT PROCESS")
print("="*50)

# 3. Test upsert một chunk đơn lẻ
if TrietHoc_child_dense_vector:
    print(f"\nTEST UPSERT 1 CHUNK:")
    try:
        test_chunk = [TrietHoc_child_dense_vector[0]]  # Chỉ lấy 1 chunk để test
        response = dense_index.upsert(
            vectors=test_chunk,
            namespace="triet-hoc-children-test"  # Test namespace riêng
        )
        print(f"SUCCESS: Test upsert thành công: {response}")
    except Exception as e:
        print(f"ERROR: Test upsert thất bại: {e}")
        print(f"Error type: {type(e)}")
        import traceback
        print(f"Traceback: {traceback.format_exc()}")

# 4. Kiểm tra namespace hiện tại
print(f"\nNAMESPACE HIỆN TẠI:")
try:
    stats = dense_index.describe_index_stats()
    for namespace, info in stats.namespaces.items():
        print(f"  {namespace}: {info.vector_count} vectors")
except Exception as e:
    print(f"ERROR: Không thể lấy stats: {e}")

DEBUG UPSERT PROCESS

TEST UPSERT 1 CHUNK:
SUCCESS: Test upsert thành công: upserted_count: 1


NAMESPACE HIỆN TẠI:
  lich-su-dang: 347 vectors
  triet-hoc: 400 vectors
  lich-su-dang-children: 1971 vectors
  triet-hoc-children-test: 0 vectors


In [20]:
# UPSERT VỚI BATCH SIZE NHỎ VÀ ERROR HANDLING
print("UPSERT CHILD CHUNKS VỚI BATCH SIZE NHỎ")
print("="*50)

# Xóa namespace test trước
try:
    dense_index.delete(delete_all=True, namespace="triet-hoc-children-test")
    print("SUCCESS: Đã xóa test namespace")
except:
    pass

# Xóa namespace chính để bắt đầu fresh
try:
    dense_index.delete(delete_all=True, namespace="triet-hoc-children")
    print("SUCCESS: Đã xóa namespace 'triet-hoc-children'")
except Exception as e:
    print(f"Warning: {e}")

print(f"Bắt đầu upsert {len(TrietHoc_child_dense_vector)} chunks...")

# Upsert với batch size nhỏ hơn
batch_size = 50  # Giảm từ 100 xuống 50
total_upserted = 0
failed_batches = []

import time

for i in range(0, len(TrietHoc_child_dense_vector), batch_size):
    batch = TrietHoc_child_dense_vector[i:i+batch_size]
    batch_num = i // batch_size + 1
    total_batches = (len(TrietHoc_child_dense_vector) + batch_size - 1) // batch_size
    
    try:
        print(f"Upsert batch {batch_num}/{total_batches} ({len(batch)} chunks)...", end=" ")
        
        # Upsert batch
        response = dense_index.upsert(
            vectors=batch,
            namespace="triet-hoc-children"
        )
        
        total_upserted += response.upserted_count
        print(f"SUCCESS: {response.upserted_count}")
        
        # Kiểm tra nếu không upsert được hết
        if response.upserted_count != len(batch):
            print(f"  WARNING: Expected {len(batch)}, got {response.upserted_count}")
            failed_batches.append(batch_num)
        
        # Thêm delay nhỏ để tránh rate limiting
        time.sleep(0.1)
        
    except Exception as e:
        print(f"ERROR: {e}")
        failed_batches.append(batch_num)
        
        # Thử upsert từng chunk một trong batch bị lỗi
        print(f"  Trying individual chunks in batch {batch_num}...")
        for j, chunk in enumerate(batch):
            try:
                individual_response = dense_index.upsert(
                    vectors=[chunk],
                    namespace="triet-hoc-children"
                )
                total_upserted += individual_response.upserted_count
                if j % 10 == 0:  # Print progress mỗi 10 chunks
                    print(f"    Chunk {j+1}/{len(batch)}")
            except Exception as individual_error:
                print(f"    ERROR: Failed chunk {j+1}: {individual_error}")

print(f"\nKẾT QUẢ:")
print(f"Total upserted: {total_upserted}/{len(TrietHoc_child_dense_vector)}")
print(f"Success rate: {total_upserted/len(TrietHoc_child_dense_vector)*100:.1f}%")

if failed_batches:
    print(f"Failed batches: {failed_batches}")
else:
    print("SUCCESS: All batches successful!")

# Kiểm tra kết quả cuối cùng
print(f"\nKIỂM TRA NAMESPACE:")
stats = dense_index.describe_index_stats()
for namespace, info in stats.namespaces.items():
    if 'triet-hoc-children' in namespace:
        print(f"  {namespace}: {info.vector_count} vectors")

if total_upserted == len(TrietHoc_child_dense_vector):
    print(f"\nHOÀN THÀNH! Đã upsert thành công tất cả {total_upserted} chunks!")
else:
    missing = len(TrietHoc_child_dense_vector) - total_upserted
    print(f"\nWARNING: Thiếu {missing} chunks. Có thể cần retry.")

UPSERT CHILD CHUNKS VỚI BATCH SIZE NHỎ
SUCCESS: Đã xóa test namespace
Bắt đầu upsert 1819 chunks...
Upsert batch 1/37 (50 chunks)... SUCCESS: 50
Upsert batch 2/37 (50 chunks)... SUCCESS: 50
Upsert batch 3/37 (50 chunks)... SUCCESS: 50
Upsert batch 4/37 (50 chunks)... SUCCESS: 50
Upsert batch 5/37 (50 chunks)... SUCCESS: 50
Upsert batch 6/37 (50 chunks)... SUCCESS: 50
Upsert batch 7/37 (50 chunks)... SUCCESS: 50
Upsert batch 8/37 (50 chunks)... SUCCESS: 50
Upsert batch 9/37 (50 chunks)... SUCCESS: 50
Upsert batch 10/37 (50 chunks)... SUCCESS: 50
Upsert batch 11/37 (50 chunks)... SUCCESS: 50
Upsert batch 12/37 (50 chunks)... SUCCESS: 50
Upsert batch 13/37 (50 chunks)... SUCCESS: 50
Upsert batch 14/37 (50 chunks)... SUCCESS: 50
Upsert batch 15/37 (50 chunks)... SUCCESS: 50
Upsert batch 16/37 (50 chunks)... SUCCESS: 50
Upsert batch 17/37 (50 chunks)... SUCCESS: 50
Upsert batch 18/37 (50 chunks)... SUCCESS: 50
Upsert batch 19/37 (50 chunks)... SUCCESS: 50
Upsert batch 20/37 (50 chunks)... S

In [21]:
# KIỂM TRA TỔNG HỢP TẤT CẢ NAMESPACE
print("KIỂM TRA TỔNG HỢP TẤT CẢ NAMESPACE")
print("="*60)

try:
    # Dense index stats
    print("DENSE INDEX STATS:")
    dense_stats = dense_index.describe_index_stats()
    for namespace, info in dense_stats.namespaces.items():
        print(f"  {namespace}: {info.vector_count} vectors")
    
    print(f"\nTotal dense vectors: {sum(info.vector_count for info in dense_stats.namespaces.values())}")
    
    # Sparse index stats
    print("\nSPARSE INDEX STATS:")
    sparse_stats = sparse_index.describe_index_stats()
    for namespace, info in sparse_stats.namespaces.items():
        print(f"  {namespace}: {info.vector_count} vectors")
    
    print(f"\nTotal sparse vectors: {sum(info.vector_count for info in sparse_stats.namespaces.values())}")
    
    # Summary theo môn học
    print("\n" + "="*60)
    print("SUMMARY THEO MÔN HỌC:")
    print("="*60)
    
    # Triết học
    triet_hoc_dense = dense_stats.namespaces.get('triet-hoc', None)
    triet_hoc_sparse = sparse_stats.namespaces.get('triet-hoc', None)
    triet_hoc_children = dense_stats.namespaces.get('triet-hoc-children', None)
    
    print("TRIẾT HỌC:")
    print(f"  - Dense (Hybrid): {triet_hoc_dense.vector_count if triet_hoc_dense else 0} vectors")
    print(f"  - Sparse (Hybrid): {triet_hoc_sparse.vector_count if triet_hoc_sparse else 0} vectors")
    print(f"  - Children (Parent Retrieval): {triet_hoc_children.vector_count if triet_hoc_children else 0} vectors")
    
    # Lịch sử Đảng
    lsd_dense = dense_stats.namespaces.get('lich-su-dang', None)
    lsd_sparse = sparse_stats.namespaces.get('lich-su-dang', None)
    lsd_children = dense_stats.namespaces.get('lich-su-dang-children', None)
    
    print("\nLỊCH SỬ ĐẢNG:")
    print(f"  - Dense (Hybrid): {lsd_dense.vector_count if lsd_dense else 0} vectors")
    print(f"  - Sparse (Hybrid): {lsd_sparse.vector_count if lsd_sparse else 0} vectors")
    print(f"  - Children (Parent Retrieval): {lsd_children.vector_count if lsd_children else 0} vectors")
    
    # Kiểm tra completeness
    print("\n" + "="*60)
    print("KIỂM TRA ĐỘ HOÀN CHỈNH:")
    print("="*60)
    
    expected_triet_hoc_dense = len(TrietHoc_dense_vector)
    expected_triet_hoc_sparse = len(TrietHoc_sparse_vector)
    expected_triet_hoc_children = len(TrietHoc_child_dense_vector)
    
    expected_lsd_dense = len(LSD_dense_vector)
    expected_lsd_sparse = len(LSD_sparse_vector)
    expected_lsd_children = len(LSD_child_dense_vector)
    
    print("TRIẾT HỌC:")
    print(f"  - Dense: {triet_hoc_dense.vector_count if triet_hoc_dense else 0}/{expected_triet_hoc_dense} ({'SUCCESS' if triet_hoc_dense and triet_hoc_dense.vector_count == expected_triet_hoc_dense else 'FAILED'})")
    print(f"  - Sparse: {triet_hoc_sparse.vector_count if triet_hoc_sparse else 0}/{expected_triet_hoc_sparse} ({'SUCCESS' if triet_hoc_sparse and triet_hoc_sparse.vector_count == expected_triet_hoc_sparse else 'FAILED'})")
    print(f"  - Children: {triet_hoc_children.vector_count if triet_hoc_children else 0}/{expected_triet_hoc_children} ({'SUCCESS' if triet_hoc_children and triet_hoc_children.vector_count == expected_triet_hoc_children else 'FAILED'})")
    
    print("\nLỊCH SỬ ĐẢNG:")
    print(f"  - Dense: {lsd_dense.vector_count if lsd_dense else 0}/{expected_lsd_dense} ({'SUCCESS' if lsd_dense and lsd_dense.vector_count == expected_lsd_dense else 'FAILED'})")
    print(f"  - Sparse: {lsd_sparse.vector_count if lsd_sparse else 0}/{expected_lsd_sparse} ({'SUCCESS' if lsd_sparse and lsd_sparse.vector_count == expected_lsd_sparse else 'FAILED'})")
    print(f"  - Children: {lsd_children.vector_count if lsd_children else 0}/{expected_lsd_children} ({'SUCCESS' if lsd_children and lsd_children.vector_count == expected_lsd_children else 'FAILED'})")
    
except Exception as e:
    print(f"ERROR: Error getting stats: {e}")
    import traceback
    print(f"Traceback: {traceback.format_exc()}")

print("\nUPSERT PROCESS COMPLETED!")
print("Tất cả dữ liệu đã được upsert lên Pinecone!")

KIỂM TRA TỔNG HỢP TẤT CẢ NAMESPACE
DENSE INDEX STATS:
  lich-su-dang-children: 1971 vectors
  triet-hoc: 400 vectors
  triet-hoc-children: 0 vectors
  lich-su-dang: 347 vectors

Total dense vectors: 2718

SPARSE INDEX STATS:
  lich-su-dang: 347 vectors
  triet-hoc: 400 vectors

Total sparse vectors: 747

SUMMARY THEO MÔN HỌC:
TRIẾT HỌC:
  - Dense (Hybrid): 400 vectors
  - Sparse (Hybrid): 400 vectors
  - Children (Parent Retrieval): 0 vectors

LỊCH SỬ ĐẢNG:
  - Dense (Hybrid): 347 vectors
  - Sparse (Hybrid): 347 vectors
  - Children (Parent Retrieval): 1971 vectors

KIỂM TRA ĐỘ HOÀN CHỈNH:
TRIẾT HỌC:
  - Dense: 400/400 (SUCCESS)
  - Sparse: 400/400 (SUCCESS)
  - Children: 0/1819 (FAILED)

LỊCH SỬ ĐẢNG:
  - Dense: 347/347 (SUCCESS)
  - Sparse: 347/347 (SUCCESS)
  - Children: 1971/1971 (SUCCESS)

UPSERT PROCESS COMPLETED!
Tất cả dữ liệu đã được upsert lên Pinecone!
